<a href="https://colab.research.google.com/github/ElleuchMed2022/ML-programming-/blob/main/camem_Bert_poste1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

In [2]:
!pip install SentencePiece

In [3]:
!nvidia-smi

Sat Mar 12 09:49:53 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P8    33W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
# Importing standard libraries for every machine/deep learning pipeline
import pandas as pd
import torch
from tqdm import tqdm, trange
import numpy as np


# Importing specific libraries for data prerpcessing, model archtecture choice, training and evaluation
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import CamembertTokenizer, CamembertForSequenceClassification
from transformers import AdamW
# import torch.optim as optim
# from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
# import seaborn as sns


In [5]:
from google.colab import files
uploaded = files.upload()

Saving data_5_column.xlsx to data_5_column.xlsx


In [6]:
import io 
df = pd.read_excel(io.BytesIO(uploaded["data_5_column.xlsx"]))
df.head(5)

,Unnamed: 0,Poste,Entreprise,Contrat,description,Localisation,salaire €
0,0,Tuyauteur (H/F),ARTUS INTERIM VANNES,Intérim,Notre agence ARTUS INTERIM de Vannes recherche...,\n ARTUS INTERI...,23000-30000
1,1,MECANICIEN PL (H/F),SOC GARAGE VEHICULES INDUSTRIELS,CDI,En tant que Mécanicien PL et rattaché au Chef ...,Évreux,23000-30000
2,2,Opérateur de production (H/F),Manpower SABLE SUR SARTHE,Intérim 3 mois,Manpower SABLE SUR SARTHE recherche pour son c...,NaN,23000-30000
3,3,TECHNICIEN ELECTROMECA SAV H/F,DE GRAËT CONSULTING|DE GRAET CONSULTING,CDI,Dans le cadre du développement constant de son...,NaN,23000-30000
4,4,ELECTRICIEN H/F,MENCO SAINT NAZAIRE,Intérim,"""MENCO ST NAZAIRE recrute des Électriciens con...",\n FINANCIERE M...,5000-23000


In [7]:
del df["Unnamed: 0"]

In [8]:
df.head(5)

,Poste,Entreprise,Contrat,description,Localisation,salaire €
0,Tuyauteur (H/F),ARTUS INTERIM VANNES,Intérim,Notre agence ARTUS INTERIM de Vannes recherche...,\n ARTUS INTERI...,23000-30000
1,MECANICIEN PL (H/F),SOC GARAGE VEHICULES INDUSTRIELS,CDI,En tant que Mécanicien PL et rattaché au Chef ...,Évreux,23000-30000
2,Opérateur de production (H/F),Manpower SABLE SUR SARTHE,Intérim 3 mois,Manpower SABLE SUR SARTHE recherche pour son c...,NaN,23000-30000
3,TECHNICIEN ELECTROMECA SAV H/F,DE GRAËT CONSULTING|DE GRAET CONSULTING,CDI,Dans le cadre du développement constant de son...,NaN,23000-30000
4,ELECTRICIEN H/F,MENCO SAINT NAZAIRE,Intérim,"""MENCO ST NAZAIRE recrute des Électriciens con...",\n FINANCIERE M...,5000-23000


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4463 entries, 0 to 4462
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Poste         4463 non-null   object
 1   Entreprise    4463 non-null   object
 2   Contrat       4463 non-null   object
 3   description   4463 non-null   object
 4   Localisation  3460 non-null   object
 5   salaire €     4463 non-null   object
dtypes: object(6)
memory usage: 209.3+ KB


In [10]:

df['Poste'][1]+" "+df['Entreprise'][1]

'MECANICIEN PL (H/F) SOC GARAGE VEHICULES INDUSTRIELS'

In [11]:
df=df.sample(frac=1)

In [12]:
# Defining constants
'''
epochs = 4
MAX_LEN = 192
batch_size =16
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device'''
# Defining constants
epochs = 4
MAX_LEN = 240
batch_size =16
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [13]:
df['Poste'][0][:MAX_LEN]

'Tuyauteur (H/F)'

In [14]:
# Initialize CamemBERT tokenizer
tokenizer = CamembertTokenizer.from_pretrained('camembert-base',do_lower_case=True)

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/508 [00:00<?, ?B/s]

In [15]:
# Creates list of texts and labels
'''
text=[]
text0 = df['Poste'].values.tolist()
text1 = df['description'].values.tolist()
#text2 = df['Localisation'].values.tolist()
labels = df['salaire €'].values.tolist()

for i in  range(df.shape[0]):
  m=text0[i]+" "+text1[i]
  text.append(m)'''
  # Creates list of texts and labels
text=[]
text0 = df['Poste'].values.tolist()
text1 = df['description'].values.tolist()

labels1 = df['salaire €'].values.tolist()
m=[]

for i in  range(df.shape[0]):
  m.append(text1[i].replace("\n"," "))
  


text=m[:4000]
text_test=m[4000:]
labels=labels1[:4000]
labels_test=labels1[4000:]



**from** sklearn.preprocessing import OneHotEncoder
LE = OneHotEncoder(handle_unknown='ignore')
labels = LE.fit_transform(labels)


In [16]:
from sklearn.preprocessing import LabelEncoder
LE = LabelEncoder()
labels = LE.fit_transform(labels)

In [17]:
labels

array([0, 0, 4, ..., 4, 1, 1])

In [18]:
#user tokenizer to convert sentences into tokenizer
input_ids  = [tokenizer.encode(sent,add_special_tokens=True,max_length=MAX_LEN) for sent in text]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [19]:
# Pad our input tokens
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

In [20]:
# Create attention masks
attention_masks = []
# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]  
    attention_masks.append(seq_mask)

In [21]:
# Use train_test_split to split our data into train and validation sets for training
train_inputs, validation_inputs, train_labels, validation_labels, train_masks, validation_masks = train_test_split(input_ids, labels, attention_masks,
                                                            random_state=42, test_size=0.1)

In [22]:
# Convert all of our data into torch tensors, the required datatype for our model
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

In [23]:
# Create an iterator of our data with torch DataLoader. This helps save on memory during training because, unlike a for loop, 
# with an iterator the entire dataset does not need to be loaded into memory

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [24]:
# Load BertForSequenceClassification, the pretrained BERT model with a single linear classification layer on top. 
model = CamembertForSequenceClassification.from_pretrained("camembert-base", num_labels=5)
model.to(device)

Downloading:   0%|          | 0.00/424M [00:00<?, ?B/s]

Some weights of the model checkpoint at camembert-base were not used when initializing CamembertForSequenceClassification: ['roberta.pooler.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.out_proj.bias

CamembertForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(32005, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (Laye

In [25]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=2e-5, eps=10e-8)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [26]:
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [27]:
# Store our loss and accuracy for plotting if we want to visualize training evolution per epochs after the training process
train_loss_set = []

In [28]:
!pip install tensorflow-addons
import tensorflow_addons as tfa

     |████████████████████████████████| 1.1 MB 5.3 MB/s 


In [29]:
for _ in trange(epochs, desc="Epoch"):  
    # Tracking variables for training
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
  
    # Train the model
    model.train()
    for step, batch in enumerate(train_dataloader):
        # Add batch to device CPU or GPU
        batch = tuple(t.to(device) for t in batch)
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        # Clear out the gradients (by default they accumulate)
        optimizer.zero_grad()
        # Forward pass
        outputs = model(b_input_ids,token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        # Get loss value
        loss = outputs[0]
        # Add it to train loss list
        train_loss_set.append(loss.item())    
        # Backward pass
        loss.backward()
        # Update parameters and take a step using the computed gradient
        optimizer.step()
    
        # Update tracking variables
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1

    print("Train loss: {}".format(tr_loss/nb_tr_steps))
    
    


    # Tracking variables for validation
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    # Validation of the model
    model.eval()
    # Evaluate data for one epoch
    for batch in validation_dataloader:
        # Add batch to device CPU or GPU
        batch = tuple(t.to(device) for t in batch)
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        # Telling the model not to compute or store gradients, saving memory and speeding up validation
        with torch.no_grad():
            # Forward pass, calculate logit predictions
            outputs =  model(b_input_ids,token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
            loss, logits = outputs[:2]
    
        # Move logits and labels to CPU if GPU is used
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
    

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Train loss: 1.1491616008016798


Epoch:  25%|██▌       | 1/4 [05:10<15:32, 310.73s/it]

Validation Accuracy: 0.64
Train loss: 0.753906236357159


Epoch:  50%|█████     | 2/4 [10:20<10:20, 310.35s/it]

Validation Accuracy: 0.785
Train loss: 0.5171365741888682


Epoch:  75%|███████▌  | 3/4 [15:30<05:10, 310.16s/it]

Validation Accuracy: 0.825
Train loss: 0.3595763210786713


Epoch: 100%|██████████| 4/4 [20:40<00:00, 310.23s/it]

Validation Accuracy: 0.8525


In [30]:
len(text_test)

463

In [31]:
# Encode the comments
tokenized_comments_ids = [tokenizer.encode(comment,add_special_tokens=True,max_length=MAX_LEN) for comment in text_test]
# Pad the resulted encoded comments
tokenized_comments_ids = pad_sequences(tokenized_comments_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

# Create attention masks 
attention_masks = []
for seq in tokenized_comments_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask)

prediction_inputs = torch.tensor(tokenized_comments_ids)
prediction_masks = torch.tensor(attention_masks)

In [32]:
# Apply the finetuned model (Camembert)
flat_pred = []
with torch.no_grad():
    # Forward pass, calculate logit predictions
    outputs =  model(prediction_inputs.to(device),token_type_ids=None, attention_mask=prediction_masks.to(device))
    logits = outputs[0]
    logits = logits.detach().cpu().numpy() 
    flat_pred.extend(np.argmax(logits, axis=1).flatten())



In [33]:
from sklearn.preprocessing import LabelEncoder
labels_test=labels1[4000:]
le = LabelEncoder()
labels_test = LE.fit_transform(labels_test)

In [34]:

l=[]
for i in range(len(flat_pred)):
    
    l.append(flat_pred[i])


In [35]:
from sklearn.metrics import accuracy_score
print(accuracy_score(labels_test,l))

0.8725701943844493


In [36]:
from sklearn.metrics import f1_score

f1 = f1_score(y_true=labels_test, y_pred=l,average="macro")
print(f1)

0.8663367832266726


In [37]:
from sklearn.metrics import classification_report
print(classification_report(labels_test, l))

              precision    recall  f1-score   support

           0       0.92      0.90      0.91       162
           1       0.83      0.88      0.85        99
           2       0.82      0.87      0.84        91
           3       0.89      0.83      0.86        77
           4       0.88      0.85      0.87        34

    accuracy                           0.87       463
   macro avg       0.87      0.87      0.87       463
weighted avg       0.87      0.87      0.87       463



In [38]:
labels_test.tolist().count(1)

99

In [39]:
l.count(1)

105